In [1]:
# read the data
from azureml.core import Workspace, Dataset

subscription_id = '353b9ff5-223a-4dc4-853e-825bc329d30c'
resource_group = 'rgbubble'
workspace_name = 'pikachu'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='cancerdataset')
data = dataset.to_pandas_dataframe()

In [2]:
# ID column was to be set as index

data = data.set_index('id')


In [3]:
logic = lambda val: 1 if val=='M' else 0
data_encoded = data
data_encoded['diagnosis'] = data['diagnosis'].map(logic)

In [5]:
data_encoded.tail()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
id,,,,,,,,,,,,,,,,,,,,,
926424,1,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
926682,1,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
926954,1,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
927241,1,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400
92751,0,7.76,24.54,47.92,181.0,0.05263,0.04362,0.00000,0.00000,0.1587,...,9.456,30.37,59.16,268.6,0.08996,0.06444,0.0000,0.0000,0.2871,0.07039


In [6]:
# split the data
from sklearn.model_selection import train_test_split
y = data_encoded['diagnosis']
x = data_encoded.iloc[:, 1:]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)

In [18]:
# Like autoML, we will try out multiple algorithms and see if any algo outperforms all the others!

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

In [19]:
mymodels = {
    "Logistic Regression" : LogisticRegression(),
    "DTree5" : DecisionTreeClassifier(max_depth=5),
    "DTree10" : DecisionTreeClassifier(max_depth=10),
    "DTree20" : DecisionTreeClassifier(max_depth=20),
    "DTree50" : DecisionTreeClassifier(max_depth=50),
    "DTree100" : DecisionTreeClassifier(max_depth=100),
    "Sherlock" : GaussianNB(),
    "RandomForestDefault100": RandomForestClassifier(),
    "RandomForestDefault10": RandomForestClassifier(n_estimators=10),
    "RandomForestDefault200": RandomForestClassifier(n_estimators=200),
    "RandomForestDefault50": RandomForestClassifier(n_estimators=50),
    "RandomForestDefault500": RandomForestClassifier(n_estimators=500),
    "Neighborhood": KNeighborsClassifier(),
    "SVC" : SVC(),
    "LinearSVC" : LinearSVC()
}

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
for algo_name, algo in mymodels.items():
    print('Algorithm in consideration = ' + algo_name)
    algo.fit(xtrain, ytrain)
    p = algo.predict(xtest)
    ascore = accuracy_score(p, ytest)
    pscore = precision_score(p, ytest)
    rscore = recall_score(p, ytest)
    print('Accuracy = ' + str(ascore))
    print('Precision = ' + str(pscore))
    print('Recall = ' + str(rscore))
    print('***************************************')

Algorithm in consideration = Logistic Regression
Accuracy = 0.9385964912280702
Precision = 0.8541666666666666
Recall = 1.0
***************************************
Algorithm in consideration = DTree5
Accuracy = 0.9122807017543859
Precision = 0.875
Recall = 0.9130434782608695
***************************************
Algorithm in consideration = DTree10
Accuracy = 0.9210526315789473
Precision = 0.9166666666666666
Recall = 0.8979591836734694
***************************************
Algorithm in consideration = DTree20
Accuracy = 0.9298245614035088
Precision = 0.9166666666666666
Recall = 0.9166666666666666
***************************************
Algorithm in consideration = DTree50
Accuracy = 0.9210526315789473
Precision = 0.8958333333333334
Recall = 0.9148936170212766
***************************************
Algorithm in consideration = DTree100
Accuracy = 0.9122807017543859
Precision = 0.8958333333333334
Recall = 0.8958333333333334
***************************************
Algorithm in conside

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy = 0.956140350877193
Precision = 0.9166666666666666
Recall = 0.9777777777777777
***************************************
Algorithm in consideration = RandomForestDefault10
Accuracy = 0.9385964912280702
Precision = 0.875
Recall = 0.9767441860465116
***************************************
Algorithm in consideration = RandomForestDefault200
Accuracy = 0.956140350877193
Precision = 0.9375
Recall = 0.9574468085106383
***************************************
Algorithm in consideration = RandomForestDefault50
Accuracy = 0.9473684210526315
Precision = 0.9166666666666666
Recall = 0.9565217391304348
***************************************
Algorithm in consideration = RandomForestDefault500
Accuracy = 0.9473684210526315
Precision = 0.9166666666666666
Recall = 0.9565217391304348
***************************************
Algorithm in consideration = Neighborhood
Accuracy = 0.9473684210526315
Precision = 0.9166666666666666
Recall = 0.9565217391304348
***************************************
Algor

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [41]:
# just a better way of writing what we wrong above with lesser manual checks 


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

best_acc = 0
best_p = 0
best_r = 0
best_f1 = 0

best_acc_model = 0
best_p_model = 0
best_r_model = 0
best_f1_model = 0

for algo_name, algo in mymodels.items():
    print('Algorithm in consideration = ' + algo_name)
    algo.fit(xtrain, ytrain)
    p = algo.predict(xtest)
    ascore = accuracy_score(p, ytest)
    pscore = precision_score(p, ytest)
    rscore = recall_score(p, ytest)
    f1s = f1_score(p, ytest)
    print('F1 Score = ' + str(f1s))
    print('Accuracy = ' + str(ascore))
    print('Precision = ' + str(pscore))
    print('Recall = ' + str(rscore))
    
    if(ascore >= best_acc and ascore != 1.0):
        best_acc = ascore
        best_acc_model = { 'Name': algo_name, 'Model' : algo}
    
    if(pscore >= best_p and pscore != 1.0):
        best_p = pscore
        best_p_model = { 'Name': algo_name, 'Model' : algo}
        
    if(rscore >= best_r and rscore != 1.0):
        best_r = rscore
        best_r_model = { 'Name': algo_name, 'Model' : algo}
    
    if(f1s >= best_f1):
        best_f1 = f1s
        best_f1_model = { 'Name': algo_name, 'Model' : algo}
    
    
    print('***************************************')

Algorithm in consideration = Logistic Regression
F1 Score = 0.9213483146067416
Accuracy = 0.9385964912280702
Precision = 0.8541666666666666
Recall = 1.0
***************************************
Algorithm in consideration = DTree5
F1 Score = 0.8817204301075269
Accuracy = 0.9035087719298246
Precision = 0.8541666666666666
Recall = 0.9111111111111111
***************************************
Algorithm in consideration = DTree10
F1 Score = 0.8979591836734694
Accuracy = 0.9122807017543859
Precision = 0.9166666666666666
Recall = 0.88
***************************************
Algorithm in consideration = DTree20
F1 Score = 0.9278350515463918
Accuracy = 0.9385964912280702
Precision = 0.9375
Recall = 0.9183673469387755
***************************************
Algorithm in consideration = DTree50
F1 Score = 0.9292929292929293
Accuracy = 0.9385964912280702
Precision = 0.9583333333333334
Recall = 0.9019607843137255
***************************************
Algorithm in consideration = DTree100
F1 Score = 0

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


F1 Score = 0.9361702127659574
Accuracy = 0.9473684210526315
Precision = 0.9166666666666666
Recall = 0.9565217391304348
***************************************
Algorithm in consideration = RandomForestDefault10
F1 Score = 0.924731182795699
Accuracy = 0.9385964912280702
Precision = 0.8958333333333334
Recall = 0.9555555555555556
***************************************
Algorithm in consideration = RandomForestDefault200
F1 Score = 0.946236559139785
Accuracy = 0.956140350877193
Precision = 0.9166666666666666
Recall = 0.9777777777777777
***************************************
Algorithm in consideration = RandomForestDefault50
F1 Score = 0.9361702127659574
Accuracy = 0.9473684210526315
Precision = 0.9166666666666666
Recall = 0.9565217391304348
***************************************
Algorithm in consideration = RandomForestDefault500
F1 Score = 0.9361702127659574
Accuracy = 0.9473684210526315
Precision = 0.9166666666666666
Recall = 0.9565217391304348
***************************************
Al

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [42]:
best_acc_model, best_acc

({'Name': 'RandomForestDefault200',
  'Model': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                         criterion='gini', max_depth=None, max_features='auto',
                         max_leaf_nodes=None, max_samples=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=2,
                         min_weight_fraction_leaf=0.0, n_estimators=200,
                         n_jobs=None, oob_score=False, random_state=None,
                         verbose=0, warm_start=False)},
 0.956140350877193)

In [43]:
best_p_model, best_p

({'Name': 'DTree100',
  'Model': DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                         max_depth=100, max_features=None, max_leaf_nodes=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=2,
                         min_weight_fraction_leaf=0.0, presort='deprecated',
                         random_state=None, splitter='best')},
 0.9583333333333334)

In [44]:
best_r_model, best_r

({'Name': 'RandomForestDefault200',
  'Model': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                         criterion='gini', max_depth=None, max_features='auto',
                         max_leaf_nodes=None, max_samples=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=2,
                         min_weight_fraction_leaf=0.0, n_estimators=200,
                         n_jobs=None, oob_score=False, random_state=None,
                         verbose=0, warm_start=False)},
 0.9777777777777777)

In [45]:
best_f1_model, best_f1

({'Name': 'RandomForestDefault200',
  'Model': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                         criterion='gini', max_depth=None, max_features='auto',
                         max_leaf_nodes=None, max_samples=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=2,
                         min_weight_fraction_leaf=0.0, n_estimators=200,
                         n_jobs=None, oob_score=False, random_state=None,
                         verbose=0, warm_start=False)},
 0.946236559139785)